# Animate simulation data

Python version of `sim_data_BM1/animate_sim_data.m`.

Work in progress for [NTHMP Debris Modeling Benchmarking Workshop](http://tsunamiworkshop.org/index.html), May 2023, by the [UW Tsunami Modeling Group](http://depts.washington.edu/ptha/).

Read in the data provided in `sim_data_BM1` and make an animation of the results, based on the Matlab script `sim_data_BM1/animate_sim_data.m`.

Requires the `sim_data_BM1` directory available from http://tsunamiworkshop.org/problems/prob1.html

Uses the `animation_tools.py` module from Clawpack, [available in github](https://github.com/clawpack/visclaw/blob/master/src/python/visclaw/animation_tools.py).

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os
import netCDF4
from scipy.interpolate import RegularGridInterpolator
import animation_tools
from IPython.display import HTML

In [ ]:
sim_data_dir = 'sim_data_BM1'

In [ ]:
eta_fname = os.path.join(sim_data_dir, 'zeta.nc')
f = netCDF4.Dataset(eta_fname, 'r')
time_numerical = array(f.variables['time'])
x_numerical = array(f.variables['x'])
zeta = squeeze(array(f.variables['zeta']))  # squeeze to get rid of y-dimension index
depth = array(f.variables['depth'])

# numerical shift values - numerical domain was not same as physical domain
x_shift=-10.  # added 10-m to offshore boundary to create wave
t_shift=20-.74  # cut off first 20 ish seconds since nothing happened

time=time_numerical+t_shift
x=x_numerical+x_shift

In [ ]:
velo_fname = os.path.join(sim_data_dir, 'velo.nc')
f = netCDF4.Dataset(velo_fname, 'r')
u_vel = squeeze(array(f.variables['u_velo']))  # squeeze to get rid of y-dimension index

In [ ]:
blvs_fname = os.path.join(sim_data_dir, 'blvs.nc')
f = netCDF4.Dataset(blvs_fname, 'r')
wet_dry = squeeze(array(f.variables['boundary_boolean']))  # wet/dry surface, =0 when wet, =99
nu_breaking = squeeze(array(f.variables['eddy_viscosity']))  # breaking eddy viscosity, m^2/s

In [ ]:
nu_breaking = where(wet_dry==99, nan, nu_breaking)
zeta = where(wet_dry==99, nan, zeta)
u_vel = where(wet_dry==99, nan, u_vel)

In [ ]:
nt = len(time)
figs = []  # list to accumulate figures for animation
for n in range(0,nt,100):
    fig = figure(figsize=(10,6))
    subplot(2,1,1)
    plot(x,-depth,'g', label='Bottom profile')
    plot(x,zeta[n,:],'b', label='Water Surface profile')

    axis([0, 44, -1, 0.5])
    #xlabel('Distance from Wavemaker (m)')
    ylabel('Elevation wrt SWL (m)')
    #title('Water Elevation at Time %.1f sec ' % time[n])
    legend(loc='lower right', fontsize=11)
    title('Time t = %.3f seconds' % time[n], fontsize=13)
    grid(True)

    subplot(2,1,2)
    plot(x,u_vel[n,:],'b',label='Cross-shore velocity (m/s)')
    axis([0, 44, -2, 3])
    #ylabel('Cross-shore Velocity (m/s)')
    
    plot(x,3*nu_breaking[n,:],'r',label='3 * Eddy viscosity (m^2/s)')
    #axis([0 44 0 1])
    xlabel('Distance from Wavemaker (m)', fontsize=13)
    #ylabel('Breaking Eddy Viscosity (m^2/s)')
    #title('Cross-shore Velocity and Eddy Viscosity at Time %.1f sec ' % time[n])
    legend(loc='upper left', fontsize=11)
    grid(True)
    figs.append(fig)
    close(fig)
    
anim = animation_tools.animate_figs(figs, figsize=(10,6))
HTML(anim.to_jshtml())